In [1]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import scanpy as sc
import tifffile
import torch
import torchvision.transforms.functional as TF
import yaml
from einops import rearrange

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
from mushroom.data.multiplex import extract_ome_tiff, get_ome_tiff_channels, make_pseudo

In [5]:
run_dir = '/data/estorrs/mushroom/data/projects/subclone'
Path(run_dir).mkdir(parents=True, exist_ok=True)

In [ ]:
data_map = {
    'HT397B1': {
        'order': [
            'HT397B1-U1',
            'HT397B1-U2',
            'HT397B1-U12',
            'HT397B1-U21',
            'HT397B1-U22',
            'HT397B1-U31'
        ],
        'data': {
            'visium': {
                'HT397B1-U1': '/data/spatial_transcriptomics/spaceranger_outputs/breast/HT397B1-S1H3A1U1/',
                'HT397B1-U21': '/data/spatial_transcriptomics/spaceranger_outputs/breast/HT397B1-S1H3A1U21/',
            },
            'he': {
                'HT397B1-U1':  '/data/spatial_transcriptomics/highres_images/breast/A1_HT397B1-S1H3A1U1.tif',
                'HT397B1-U21': '/data/spatial_transcriptomics/highres_images/breast/B1_HT397B1-S1H3A1U21.tif',
            },
            'multiplex': {
                'HT397B1-U2': '/data/multiplex_imaging/codex/htan/brca/031623_BRCA_HT397B1-U2/level_2/HT397B1_U2_03162023.ome.tiff',
                'HT397B1-U12': '/data/multiplex_imaging/codex/htan/brca/03172023_BRCA_HT397B1-U12/level_2/HT397B1_U12_03172023.ome.tiff',
                'HT397B1-U22': '/data/multiplex_imaging/codex/htan/brca/041223_BRCA_HT397B1-S1H3A1-U22/level_2/HT397B1_S1H1A3U22_04122023.ome.tiff',
                'HT397B1-U31': '/data/multiplex_imaging/codex/htan/brca/040623_BRCA_HT397B1-U31/level_2/HT397B1_S1H1A3U31_04062023.ome.tiff',
            }
        }
    },
    # look at HT271B1-S1H3Fc2U1Z1Bs1 for codex alignment
    #!!! 413 has codex
    # I think 264 does, but not in validation cohort
    '': {
        'order': [
        ],
        'data': {
            'visium': {
            },
            'he': {
            },
            'multiplex': {
            }
        }
    },
        '': {
        'order': [
        ],
        'data': {
            'visium': {
            },
            'he': {
            },
            'multiplex': {
            }
        }
    },
        '': {
        'order': [
        ],
        'data': {
            'visium': {
            },
            'he': {
            },
            'multiplex': {
            }
        }
    },
    '': {
        'order': [
        ],
        'data': {
            'visium': {
            },
            'he': {
            },
            'multiplex': {
            }
        }
    }
}

In [ ]:
# spaceranger output directories
section_to_visium = {
    'HT397B1-U1': '/data/spatial_transcriptomics/spaceranger_outputs/breast/HT397B1-S1H3A1U1/',
    'HT397B1-U21': '/data/spatial_transcriptomics/spaceranger_outputs/breast/HT397B1-S1H3A1U21/',
}

# hi res H&E 
section_to_he = {
    'HT397B1-U1':  '/data/spatial_transcriptomics/highres_images/breast/A1_HT397B1-S1H3A1U1.tif',
    'HT397B1-U21': '/data/spatial_transcriptomics/highres_images/breast/B1_HT397B1-S1H3A1U21.tif',
}

# codex
section_to_multiplex = {
    'HT397B1-U2': '/data/multiplex_imaging/codex/htan/brca/031623_BRCA_HT397B1-U2/level_2/HT397B1_U2_03162023.ome.tiff',
    'HT397B1-U12': '/data/multiplex_imaging/codex/htan/brca/03172023_BRCA_HT397B1-U12/level_2/HT397B1_U12_03172023.ome.tiff',
    'HT397B1-U22': '/data/multiplex_imaging/codex/htan/brca/041223_BRCA_HT397B1-S1H3A1-U22/level_2/HT397B1_S1H1A3U22_04122023.ome.tiff',
    'HT397B1-U31': '/data/multiplex_imaging/codex/htan/brca/040623_BRCA_HT397B1-U31/level_2/HT397B1_S1H1A3U31_04062023.ome.tiff',
}